# Proyecto Final Lab Inteligencia (Imagenes): IRP 

In [2]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.spatial import distance as dist
import argparse
import glob
from skimage.feature import hog
from skimage import exposure

In [132]:
#Realizar la carga de las listas usando numpy.load
#Guardar datos.
#5 Divisiones
nombres5 = np.load('nombres5.npy')
clases5 = np.load('clases5.npy')
histos5 = np.load('histos5.npy')
nombresQ5 = np.load('nombresQ5.npy')
clasesQ5 = np.load('clasesQ5.npy')
compQ5 = np.load('compQ5.npy')
#Divisiones con HOG
nombresHOG = np.load('nombresHoG.npy')
clasesHOG = np.load('clasesHoG.npy')
histosHOG = np.load('histosHoG.npy')
nombresQueryAlt = np.load('nombresQueryAlt.npy')
compQueryAlt = np.load('compQueryAlt.npy')
clasesQueryAlt = np.load('clasesQueryAlt.npy')

In [133]:
compQ5.shape

(150, 407)

In [134]:
compQueryAlt.shape

(150, 407)

## 8) IRP

### 8.1) Implementación de IRP.

In [145]:
#Funcion que ordene de acuerdo a los valores de cercania
def ordenarIRP(indices,vecComparaciones):
    #Orden
    orden = np.argsort(vecComparaciones)
    #Ordenar los indices de acuerdo al orden
    indexOrdenado = np.zeros_like(indices)
    count = 0
    for i in orden:
        indexOrdenado[count] = indices[i]
        count += 1
    return indexOrdenado

#Funcion que calcule los nuevos valores IRP para la combinación de los métodos.
def valoresIRPIndividual(vecComp1,vecComp2):
    largoComp = len(vecComp1)
    #Nombres de imagen
    name1 = np.arange(largoComp) + 1
    name2 = np.arange(largoComp) + 1
    #Ordenar y obtener el orden de índices.
    ind1 = ordenarIRP(name1,vecComp1)
    ind2 = ordenarIRP(name2,vecComp2)
    #Obtener los valores de EIRP asociados a cada imagen en orden. 
    valoresIRP = np.zeros_like(vecComp1)
    count = 0
    for i in name1:
        pos1 = np.where(ind1 == i)[0][0] + 1
        pos2 = np.where(ind2 == i)[0][0] + 1
        valorIRP = 1/((1/pos1)+(1/pos2))
        valoresIRP[count] = valorIRP
        count += 1
    return valoresIRP  

def ordenarClasesNombre(IRP, clases, nombres):
    #Ordenar IRP.
    orden = np.argsort(IRP)
    #Generacion de arreglos ordenados.
    clasesOrdenadas = np.zeros_like(clases)
    nombresOrdenados = np.zeros_like(nombres)
    #Ordenar clases y nombres.
    count = 0
    for i in orden:
        clasesOrdenadas[count] = clases[i]
        nombresOrdenados[count] = nombres[i]
        count += 1
    return clasesOrdenadas,nombresOrdenados
        
    
def ordenarPorIRP(matComparaciones1,matComparaciones2,clases = clases5 ,nombres = nombres5):
    #Generar nueva matriz
    newMat = np.array([])
    #Dimensiones de la matriz.
    fils,cols = matComparaciones1.shape
    #Ciclo.
    count = 0
    #Arreglos que almacenen nombres y clases ordenadas.
    matClasesOrdenadas = np.array([])
    matNombresOrdenadas = np.array([])
    while count < fils:
        vectorIRP = valoresIRPIndividual(matComparaciones1[count,:],matComparaciones2[count,:])
        classOrdenada,nameOrdenado = ordenarClasesNombre(vectorIRP, clases,nombres)
        if count == 0:
            matClasesOrdenadas = classOrdenada
            matNombresOrdenadas = nameOrdenado
        elif(count==1):
            matClasesOrdenadas = np.concatenate((np.array([matClasesOrdenadas]),np.array([classOrdenada])),axis=0)
            matNombresOrdenadas = np.concatenate((np.array([matNombresOrdenadas]),np.array([nameOrdenado])),axis=0)
        else:
            matClasesOrdenadas = np.concatenate((matClasesOrdenadas,np.array([classOrdenada])),axis=0)
            matNombresOrdenadas = np.concatenate((matNombresOrdenadas,np.array([nameOrdenado])),axis=0)
        count += 1
    return matClasesOrdenadas,matNombresOrdenadas

In [146]:
matClasesIRP,matNombreIRP = ordenarPorIRP(compQueryAlt,compQ5)

In [149]:
matClasesIRP.shape

(150, 407)

### 8.2) Cálculo de métricas. 

In [150]:
#Funcion que calcule el Rank Normalizado para las figuras.
def rankNormalizadoIndividualIRP(claseQuery,vectorClasesOrdenado):
    #Numero total de elementos en el conjunto.
    N = len(nombresHOG)  
    #Elementos del conjunto de la misma clase que el elemnto de la Query.
    Nrel = np.count_nonzero(clasesHOG == claseQuery)
    #Ordenar comparaciones y datos. Ordenamiento de acuerdo al valor del vector de comparaciones.
    #Calculo del rank normalizado.
    factor1 = 1/(N*Nrel)
    factor2 = (Nrel*(Nrel+1))/(2)
    sumaR = np.sum(np.ravel(np.argwhere(vectorClasesOrdenado==claseQuery)))
    rankNorm = factor1 * (sumaR - factor2)
    return rankNorm

#Funcion que calcule el rank normalizado promedio de todas las figuras query.
def rankNormalizadoIRP():
    #Funcion que calcule el rank promedio de todas las query.
    count = 0
    rankTot = np.array([])
    for clasesOrd in matClasesIRP:
        rank = rankNormalizadoIndividualIRP(clasesQ5[count],clasesOrd)
        rankTot = np.append(rankTot,rank)
        count += 1
    rankPromedio = np.mean(rankTot)
    return rankPromedio

In [151]:
rankNormIRP = rankNormalizadoIRP()

In [152]:
print(rankNormIRP)

0.1026995956995957


### 8.3) Visualización de resultados.

In [ ]:
#Funcion que entregue las diez mejores imagenes
def parte6(numeroImagen):    
    comparaciones = compQueryAlt[numeroImagen,:]
    comparacionesOrdenado,clasesOrdenado,nombresOrdenado = ordenarVector(comparaciones,clasesHoG,nombresHoG)
    print('Imagen Query:', nombresQueryAlt[numeroImagen])
    mejores = nombresOrdenado[0:10]
    print('Mejores 10 imagenes:', mejores)
    # Imprimir Query
    imQ = cv2.imread("img_query\\"+nombresQueryAlt[numeroImagen])
    imQ = cv2.resize(imQ,None,fx=.2, fy=.2, interpolation = cv2.INTER_CUBIC)
    cv2.imshow("imagenQuery",imQ)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # loop over the results
    for im in mejores:
        # load the result image and display it
        result = cv2.imread("img_database\\" + im)
        result = cv2.resize(result,None,fx=.2, fy=.2, interpolation = cv2.INTER_CUBIC)
        cv2.imshow(im, result)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
#Funcion que entregue las diez mejores imagenes
def parte6Pro(numeroImagen):    
    comparaciones = compQueryAlt[numeroImagen,:]
    #Ordenar
    comparacionesOrdenado,clasesOrdenado,nombresOrdenado = ordenarVector(comparaciones,clasesHoG,nombresHoG)
    mejores = nombresOrdenado[0:10]
    nameQuery = nombresQueryAlt[numeroImagen]
    claseQuery = clasesQueryAlt[numeroImagen]
    #Definicion del grafico.
    fig=plt.figure(figsize=(20, 20))
    columns = 3
    rows = 5
    #Imagen Query
    imQ = cv2.imread("img_query\\"+nombresQueryAlt[numeroImagen])
    imQ = cv2.cvtColor(imQ,cv2.COLOR_BGR2RGB)
    fig.add_subplot(rows, columns, 2)
    plt.axis('off')
    plt.title("imagen Query. Clase: " + str(clasesQueryAlt[numeroImagen]))
    plt.imshow(imQ, aspect='auto')
    #Presentación de opciones.
    for i in range(0, 10):
        imgBGR = cv2.imread("img_database\\" + mejores[i])
        img = cv2.cvtColor(imgBGR,cv2.COLOR_BGR2RGB)
        fig.add_subplot(rows, columns, i+4)
        plt.axis('off')   
        if(str(claseQuery)==str(clasesOrdenado[i])):
             plt.title("Imagen Database de clase: " + str(clasesOrdenado[i]),color='g')
        else:
             plt.title("Imagen database de tipo: " + str(clasesOrdenado[i]),color='r')
        plt.imshow(img, aspect='auto')
        
    plt.savefig('comparacionImagen' + str(numeroImagen) + 'divHog' +  '.jpg')